# Sprint23課題 深層学習スクラッチリカレントニューラルネットワーク

## 【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。

今回はバッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。

$$
a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + b\\
h_t = tanh(a_t)
$$

$a_{t}$ : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)

$h_{t}$ : 時刻tの状態・出力 (batch_size, n_nodes)

$x_{t}$ : 時刻tの入力 (batch_size, n_features)

$W_{x}$ : 入力に対する重み (n_features, n_nodes)

$h_{t−1}$ : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)

$W_{h}$ : 状態に対する重み。 (n_nodes, n_nodes)

$b$: バイアス項 (1,)


初期状態 
$h_{0}$ は全て0とすることが多いですが、任意の値を与えることも可能です。上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力$x$ は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。
分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。出力は最後のhだけを使用する場合と、全てのhを使う場合があります。



## 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。
入力$x$、初期状態$h$、重み$w_{x}$とw_h、バイアス$b$を次のようにします。

ここで配列$x$の軸はバッチサイズ、系列数、特徴量数の順番です。

In [1]:
import numpy as np
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes))
b = np.array([1])

In [2]:
def tanh(x):
        return (-2 /  (np.exp(2*x) + 1) + 1)  

In [3]:
a_0 = x[:, 0, :] @ w_x +  h @ w_h + b
h_0 = tanh(a_0)
a_1 = x[:, 1, :] @ w_x +  h_0 @ w_h + b
h_1 = tanh(a_1)
a_2 = x[:, 2, :] @ w_x +  h_1 @ w_h + b
h_2 = tanh(a_2)
h_2

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

## 【問題3】（アドバンス課題）バックプロパゲーションの実装
バックプロパゲーションを実装します。

RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。

$$
W_x^{\prime} = W_x - \alpha E(\frac{\partial L}{\partial W_x}) \\
W_h^{\prime} = W_h - \alpha E(\frac{\partial L}{\partial W_h}) \\
b^{\prime} = b - \alpha E(\frac{\partial L}{\partial b})
$$

$\alpha$ : 学習率<br>
$\frac{\partial L}{\partial W_{x}}$: $W_{x}$に関する損失 Lの勾配<br>
$\frac{\partial L}{\partial W_{h}}$: $W_{h}$に関する損失 Lの勾配<br>
$\frac{\partial L}{\partial b}$: $b$に関する損失 Lの勾配<br>
$E()$: ミニバッチ方向にベクトルの平均を計算<br>

勾配を求めるためのバックプロパゲーションの数式が以下です。
$\frac{\partial h_{t}}{\partial a_{t}} = \frac{\partial L}{\partial h_{t} }\times (1 - tanh^{2}(a_{t}))$<br>
$\frac{\partial L}{\partial b} = \frac{\partial h_{t}}{\partial a_{t}} $<br>
$\frac{\partial L}{\partial W_{x}} = x_{t}^{T} \cdot  \frac{\partial h_{t}}{\partial a_{t}}$<br>
$\frac{\partial L}{\partial W_{h}} = h_{t-1}^{T} \cdot  \frac{\partial h_{t}}{\partial a_{t}}$<br>
$\frac{\partial L}{\partial h_{t} }$ は前の時刻からの状態の誤差と出力の誤差の合計です。hは順伝播時に出力と次の層に伝わる状態双方に使われているからです。

前の時刻や層に流す誤差の数式は以下です。<br>

$\frac{\partial L}{\partial h_{t-1}} = \frac{\partial h_{t}}{\partial a_{t}} \cdot W_{h}^{T}$<br>
$\frac{\partial L}{\partial x_{t}} = \frac{\partial h_{t}}{\partial a_{t}} \cdot W_{x}^{T}$

RNN layeのpyファイルは[こちら](https://github.com/ohmorimori/diveintocode-ml/blob/master/diveintocode-term2/ml-scratch/model/layer.py)
<br>
RNN classifierのpyファイルは[こちら](https://github.com/ohmorimori/diveintocode-ml/blob/master/diveintocode-term2/ml-scratch/model/scratch_rnn_classifier.py)

In [5]:
import sys

dir_str = "../../ml-scratch/model"
if dir_str not in sys.path:
        sys.path.append(dir_str)
from scratch_rnn_classifier import ScratchRNNClassifier
y = np.array([1, 2, 3])
rnn = ScratchRNNClassifier(
    n_nodes1= 4, n_output = 3,  sigma=0.01, lr=0.001, batch_size=1, n_epochs=30
)
rnn.fit(x, y)
rnn.predict(x)


epoch:  0
process time:  0.0014162063598632812 sec
epoch:  1
process time:  0.0035130977630615234 sec
epoch:  2
process time:  0.005158185958862305 sec
epoch:  3
process time:  0.0067670345306396484 sec
epoch:  4
process time:  0.00880885124206543 sec
epoch:  5
process time:  0.014741897583007812 sec
epoch:  6
process time:  0.016381025314331055 sec
epoch:  7
process time:  0.01833319664001465 sec
epoch:  8
process time:  0.0219419002532959 sec
epoch:  9
process time:  0.023999929428100586 sec
epoch:  10
process time:  0.02598118782043457 sec
epoch:  11
process time:  0.030054092407226562 sec
epoch:  12
process time:  0.0340731143951416 sec
epoch:  13
process time:  0.036997079849243164 sec
epoch:  14
process time:  0.03983616828918457 sec
epoch:  15
process time:  0.04252195358276367 sec
epoch:  16
process time:  0.04587101936340332 sec
epoch:  17
process time:  0.0480959415435791 sec
epoch:  18
process time:  0.0505671501159668 sec
epoch:  19
process time:  0.05386805534362793 sec
ep

array([1])

動くことは確認できた。

## 【問題4】（アドバンス課題）データセットでの学習・推定
これまで使ってきたニューラルネットワークにSimpleRNNを組み込み学習させ、動くことを確認してください。

[IMDB Review Dataset | Kaggle](https://www.kaggle.com/utathya/imdb-review-dataset)

映画レビューデータセットを使用します。ベクトル化を行い、作成したRNNに入力してください。

In [8]:
#読み込み
#TBC
import pandas as pd

data = pd.read_csv('imdb_master.csv', encoding="ISO-8859-1")
data

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt
5,5,test,"A funny thing happened to me while watching ""M...",neg,10004_2.txt
6,6,test,This German horror film has to be one of the w...,neg,10005_2.txt
7,7,test,"Being a long-time fan of Japanese film, I expe...",neg,10006_2.txt
8,8,test,"""Tokyo Eyes"" tells of a 17 year old Japanese g...",neg,10007_4.txt
9,9,test,Wealthy horse ranchers in Buenos Aires have a ...,neg,10008_4.txt


In [ ]:
#To be continued